<a href="https://colab.research.google.com/github/MTajuddin4u/Assigment-04/blob/main/Python_Streamlit_Web_App_Country_Information_Cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile country_app.py
import streamlit as st
import requests
import pandas as pd
from PIL import Image
from io import BytesIO


st.title("🌍 Country Information Cards")
st.markdown("Explore detailed information about any country in the world!")


st.markdown("""
<style>
.card {
    box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    transition: 0.3s;
    border-radius: 10px;
    padding: 20px;
    margin: 15px 0;
    background-color: #f9f9f9;
}
.card:hover {
    box-shadow: 0 8px 16px 0 rgba(0,0,0,0.2);
}
.country-flag {
    max-width: 100%;
    height: auto;
    border: 1px solid #ddd;
    border-radius: 4px;
    margin-bottom: 10px;
}
</style>
""", unsafe_allow_html=True)


@st.cache_data
def get_all_countries():
    try:
        response = requests.get("https://restcountries.com/v3.1/all")
        if response.status_code == 200:
            countries = sorted([country['name']['common'] for country in response.json()])
            return countries
        return []
    except:
        return []

all_countries = get_all_countries()


st.sidebar.header("Search Options")
search_type = st.sidebar.radio(
    "Search by:",
    ["Country Name", "Continent", "Language"]
)

if search_type == "Country Name":
    selected_country = st.sidebar.selectbox(
        "Select a Country",
        all_countries
    )
    search_url = f"https://restcountries.com/v3.1/name/{selected_country}"
elif search_type == "Continent":
    continent = st.sidebar.selectbox(
        "Select Continent",
        ["Africa", "Americas", "Asia", "Europe", "Oceania"]
    )
    search_url = f"https://restcountries.com/v3.1/region/{continent.lower()}"
else:
    language = st.sidebar.selectbox(
        "Select Language",
        ["English", "Spanish", "French", "Arabic", "Chinese", "Hindi", "Portuguese"]
    )
    search_url = f"https://restcountries.com/v3.1/lang/{language.lower()}"


@st.cache_data
def get_country_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        return None
    except:
        return None

country_data = get_country_data(search_url)

if country_data:
    if search_type == "Country Name":
        countries_to_show = country_data[:1]
    else:
        countries_to_show = country_data[:10]

    for country in countries_to_show:

        st.markdown(f"<div class='card'>", unsafe_allow_html=True)

        col1, col2 = st.columns([1, 2])

        with col1:

            if 'flags' in country:
                flag_url = country['flags']['png']
                flag_response = requests.get(flag_url)
                flag_img = Image.open(BytesIO(flag_response.content))
                st.image(flag_img, caption=f"Flag of {country['name']['common']}", use_column_width=True)

        with col2:

            st.header(country['name']['common'])

            if 'capital' in country:
                st.subheader(f"Capital: {', '.join(country['capital'])}")

            info_cols = st.columns(2)

            with info_cols[0]:
                if 'region' in country:
                    st.write(f"**Region:** {country['region']}")
                if 'subregion' in country:
                    st.write(f"**Subregion:** {country['subregion']}")
                if 'population' in country:
                    st.write(f"**Population:** {country['population']:,}")

            with info_cols[1]:
                if 'languages' in country:
                    langs = list(country['languages'].values())
                    st.write(f"**Languages:** {', '.join(langs)}")
                if 'currencies' in country:
                    currencies = [f"{details['name']} ({details['symbol']})" for details in country['currencies'].values()]
                    st.write(f"**Currencies:** {', '.join(currencies)}")
                if 'timezones' in country:
                    st.write(f"**Timezones:** {', '.join(country['timezones'])}")


        if 'maps' in country and 'googleMaps' in country['maps']:
            st.markdown(f"[View on Google Maps]({country['maps']['googleMaps']})")

        st.markdown("</div>", unsafe_allow_html=True)
else:
    st.error("Failed to fetch country data. Please try again later.")


st.markdown("---")
st.markdown("Data provided by [REST Countries API](https://restcountries.com/)")

Writing country_app.py
